In [3]:

import os
import sys
from os import path
import pause

import threading
from threading import Timer, Thread
import schedule

import pandas as pd
import numpy as np

from datetime import datetime, timedelta, time
import time as ot
from dateutil.relativedelta import relativedelta

import yfinance as yf

import FinanceDataReader as fdr

import requests
from public_data_config import apisdata

from slack_message import _post_message
from get_holiday import _get_holiday

# kospi200 회사명 리스트 가져오기
from bs4 import BeautifulSoup
from urllib.request import urlopen

from selenium import webdriver
from selenium.webdriver.common.by import By

# pip install webdriver-manager  ## 항상 최신 버전의 chromedriver를 자동으로 사용
from webdriver_manager.chrome import ChromeDriverManager

In [28]:
class BaseService:
    def __init__(self):
        self.myToken = apisdata["slack"]["token"]
        self.channel = apisdata["slack"]["channel"]

        # 휴장일 데이터 가져오기
        self.mykey = apisdata["holiday"]["decoding_key"]
        self.request_url = apisdata["holiday"]["request_url"]
        self.year = "2022"
        self.market = "kospi"
        self.market_stock_dict = {
            self.market : None,
            }
        self.market_dict = None
        self.config_ndays = 20
        self.start_date = (datetime.now() - timedelta(days=40)).date().strftime("%Y-%m-%d")
        self.ko_holiday_list =  ['2022-01_01','2022-01_31','2022-02_01','2022-02_02','2022-03_01','2022-03_09','2022-05_05','2022-05_08','2022-06_01','2022-06_06','2022-08_15','2022-09_09','2022-09_10','2022-09_11','2022-09_12','2022-10_03','2022-10_09','2022-10_10','2022-12_25']

    def scheduler(self):
        today_date = datetime.now().date().strftime("%Y-%m-%d")
        try:
            holiday_list = _get_holiday(self)
        except Exception as e:
            print(e)
            holiday_list = []

        if today_date not in holiday_list:
            try:
                self._get_stock_list()

                self._post_signal_to_slack()
            except Exception as e:
                self._post_message(e)
                pass

        else:
            try:
                self._get_stock_list()
                self._post_signal_to_slack()
            except Exception as e:
                self._post_message(e)
                pass
    
    def _get_stock_list(self):
        stock_list = []
        if self.market == "kospi":
            for i in range(1, 21):
                page = i
                url = 'https://finance.naver.com/sise/entryJongmok.nhn?&page={page}'.format(page = page)
                source = urlopen(url).read()
                source = BeautifulSoup(source,'lxml')
                source = source.find_all('a',target = '_parent')
                for j in range(len(source)):
                    name = source[j].text
                    stock_list.append(name)

            kospi_df = fdr.StockListing(self.market)
            self.market_dict = kospi_df[kospi_df["Name"].isin(stock_list)].set_index("Symbol")["Name"].to_dict()
            self.market_stock_dict[self.market] = self.market_dict

        elif self.market == "sp500":
            self.market_dict = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].set_index("Symbol")["Security"].to_dict()
            self.market_stock_dict[self.market] = self.market_dict


    def _post_message(self, text):
        response = requests.post("https://slack.com/api/chat.postMessage",
            headers={"Authorization": "Bearer "+self.myToken},
            data={"channel": self.channel,"text": text}
        )
        if response.ok:
            print("Success. Send message with slack.")
        else:
            print("Fail. Send message with slack.")
            pass

    def _make_signal(self, df):
        try:
            df = df.reset_index()
            df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
            df["sma"] = df["TP"].rolling(self.config_ndays).mean()
            df["mad"] = df["TP"].rolling(self.config_ndays).apply(lambda x: pd.Series(x).mad())
            df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])
            
            df["pre1day_cci"] = df["CCI"].shift(1)
            df["trade"] = None
            df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
            df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])
            
            df["total_buy_price"] = 0
            df["shares"] = 0
            df["buy_price"] = 0
            df["sell_price"] = 0
            df["revenue"] = 0
            df["rate"] = 0

            buy_price_list = []
            for i, x in df.iterrows():
                if x["trade"] == "buy":
                    buy_price_list.append(x["Close"])
                    df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                    df.loc[i, "shares"] = len(buy_price_list)
                    df.loc[i, "buy_price"] = np.mean(buy_price_list)
                elif x["trade"] == None:
                    if i > 0:
                        df.loc[i, "shares"] = df.loc[i-1, "shares"]
                        df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
                        df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
                elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
                    first_price = buy_price_list.pop(0)
                    df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
                    df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
                    df.loc[i, "sell_price"] = df.loc[i, "Close"]
                    df.loc[i, "shares"] = len(buy_price_list)
                    if len(buy_price_list) > 0:
                        df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                        df.loc[i, "buy_price"] = np.mean(buy_price_list)
                    else:
                        df.loc[i, "total_buy_price"] = 0
                        df.loc[i, "buy_price"] = 0
            
            r_dict = {
                "symbol" : df.iloc[-1]["Symbol"],
                "date" : df.iloc[-1]["Date"],
                "trade_signal" : df.iloc[-1]["trade"],
                "price" : df.iloc[-1]["Close"]
            }
            return r_dict
            
        except Exception as e:
            # self._post_message(e)
            print(e)
            pass

    def _post_signal_to_slack(self):
        df = pd.DataFrame()
        buy_list = []
        sell_list = []
        idata= []

        if self.market == "kospi":
            for symbol in self.market_stock_dict[self.market]:
                try:
                    df = fdr.DataReader(symbol, start=self.start_date)
                    df["Symbol"] = symbol

                    r_dict = self._make_signal(df)

                    if r_dict["trade_signal"] == "buy":
                        buy_list.append(self.market_dict.get(r_dict["symbol"]))
                        print(r_dict)
                        idata.append(r_dict)
                    elif r_dict["trade_signal"] == "sell":
                        sell_list.append(self.market_dict.get(r_dict["symbol"]))
                        print(r_dict)
                        idata.append(r_dict)
                except Exception as e:
                    # self._post_message(e)
                    print(e)
                    pass

        elif self.market == "sp500":
            for symbol in self.market_stock_dict[self.market]:
                try:
                    df = yf.download(symbol, start=self.start_date, show_errors=False)
                    df["Symbol"] = symbol
                    
                    r_dict = self._make_signal(df)

                    if r_dict["trade_signal"] == "buy":

                        buy_list.append(r_dict["symbol"])
                        print(r_dict)
                        idata.append(r_dict)
                    elif r_dict["trade_signal"] == "sell":
                        sell_list.append(r_dict["symbol"])
                        print(r_dict)
                        idata.append(r_dict)

                except Exception as e:
                    # self._post_message(e)
                    print(e)
                    pass
            
        # 구매 데이터
        if len(buy_list) == 0:
            self._post_message("Today not exists buy stocks")
        if len(buy_list) > 0:
            self._post_message("Buy_stocks : %s"%(buy_list))

        # 판매 데이터
        if len(sell_list) == 0:
            self._post_message("Today not exists sell stocks")
        if len(sell_list) > 0:
            self._post_message("Sell_stocks : %s"%(sell_list))
        
        try:
            df = pd.DataFrame.from_dict(idata)
        except Exception as e:
            self._post_message(e)
            pass

        return df
        
    def _get_priority_house_jungso(self):
        driver = webdriver.Chrome(ChromeDriverManager().install())
        url = "https://www.smes.go.kr/sanhakin/websquare/wq_main.do"
        driver.get(url)
        driver.implicitly_wait(10)
        driver.find_element(By.XPATH, '//*[@id="genTopMenu_2_liTopMenu"]').click()
        driver.implicitly_wait(1)
        driver.find_element(By.XPATH, '//*[@id="genLeftMenu_3_leftMenuGrp"]').click()
        driver.implicitly_wait(1)

        idata = []
        # pagelist1_page_2
        search_date = datetime.today().date()
        str_search_date = datetime.strftime(search_date, "%Y-%m-%d")
        for page in range(1, 3):
            # print(page)
            driver.find_element(By.XPATH, '//*[@id="pagelist1_page_%s"]'%page).click()

            driver.implicitly_wait(1)
            for table_order in range (0, 10):
                title = driver.find_element_by_css_selector('#gridView1_cell_%s_0'%table_order).text
                str_start_date = driver.find_element_by_css_selector('#gridView1_cell_%s_3'%table_order).text.split(' ')[0]
                start_date = datetime.strptime(str_start_date, "%Y-%m-%d").date()
                if search_date <= start_date:
                    idict = {
                        "search_date" : str_search_date,
                        "title" : title,
                        "start_date" : str_start_date
                        }
                    idata.append(idict)
            if search_date > datetime.strptime(driver.find_element_by_css_selector('#gridView1_cell_9_3').text.split(' ')[0], "%Y-%m-%d").date():
                print(datetime.strptime(driver.find_element_by_css_selector('#gridView1_cell_9_3').text.split(' ')[0], "%Y-%m-%d").date())
                break
        driver.quit()
        if len(idata) > 0:
            text = "유효한 새로운 중소기업 장기근속자 주택 특별공급 없음"
            _post_message(self, text)
        else:
            text = idata
            _post_message(self, text)
        return idata

    def work(self):
        if __name__ == "__main__":
            self._get_stock_list()
        return self.market_stock_dict


In [31]:
market_stock_dict = BaseService().work()

In [116]:
ilist = []
for stock in market_stock_dict["kospi"]:
    symbol = stock+".KS"
    ilist.append(yf.Ticker(symbol).info)
market_info_df = pd.DataFrame(ilist)
market_info_df.head()

{'sector': 'Consumer Defensive', 'longBusinessSummary': 'BGF retail CO., LTD. operates convenience stores in South Korea. It also engages in franchising business. In addition, the company manufactures and distributes fresh food products, including lunch boxes, rice balls, sushi rolls, sandwiches, and hamburgers; and other items. BGF retail CO., LTD. was incorporated in 1994 and is based in Seoul, South Korea.', 'city': 'Seoul', 'phone': '82 1 577 8007', 'country': 'South Korea', 'companyOfficers': [], 'website': 'https://www.bgfretail.com', 'maxAge': 1, 'address1': 'BGF Building', 'industry': 'Grocery Stores', 'address2': '405 Taeheran-ro Samsung-dong Gangnam-ku', 'ebitdaMargins': 0.08492, 'profitMargins': 0.02177, 'grossMargins': 0.17218, 'operatingCashflow': 538147651584, 'revenueGrowth': 0.12, 'operatingMargins': 0.02853, 'ebitda': 575849431040, 'targetLowPrice': 174000, 'recommendationKey': 'buy', 'grossProfits': 1167595525590, 'freeCashflow': 205686718464, 'targetMedianPrice': 210

In [60]:
kospi_df = pd.DataFrame(ilist)
kospi_df.head()

,sector,longBusinessSummary,city,phone,country,companyOfficers,website,maxAge,address1,industry,...,regularMarketPrice,preMarketPrice,logo_url,fullTimeEmployees,zip,fax,underlyingSymbol,underlyingExchangeSymbol,headSymbol,uuid
0,Consumer Defensive,"BGF retail CO., LTD. operates convenience stor...",Seoul,82 1 577 8007,South Korea,[],https://www.bgfretail.com,1,BGF Building,Grocery Stores,...,178500,None,https://logo.clearbit.com/bgfretail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Financial Services,"BNK Financial Group Inc., together with its su...",Busan,82 5 1620 3000,South Korea,[],https://www.bnkfg.com,1,"30, Munhyeongeumyeong-ro",Banks—Regional,...,7970,None,https://logo.clearbit.com/bnkfg.com,89.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Industrials,CJ Corporation engages in the food and food se...,Seoul,82 2 726 8114,South Korea,[],https://www.cj.net,1,CJ THE CENTER,Conglomerates,...,85800,None,https://logo.clearbit.com/cj.net,NaN,04637,NaN,NaN,NaN,NaN,NaN
3,Communication Services,"CJ CGV Co., Ltd. engages in the screening and ...",Seoul,82 2 371 6660,South Korea,[],https://www.cgv.co.kr,1,I-Park Mall,Entertainment,...,27350,None,https://logo.clearbit.com/cgv.co.kr,NaN,NaN,82 2 2017 1420,NaN,NaN,NaN,NaN
4,Industrials,CJ Logistics Corporation provides integrated t...,Seoul,82 1 588 1255,South Korea,[],https://www.cjlogistics.com,1,CJ Logistics Building,Integrated Freight & Logistics,...,123000,None,https://logo.clearbit.com/cjlogistics.com,NaN,04513,NaN,NaN,NaN,NaN,NaN


In [120]:
kospi_df['sharesOutstanding']

0       17275000.0
1      324432000.0
2       27058500.0
3       40806200.0
4       19944900.0
          ...     
195     19909400.0
196      4468020.0
197      4315690.0
198     92606800.0
199     60095800.0
Name: sharesOutstanding, Length: 200, dtype: float64

In [122]:
kospi_df[kospi_df["symbol"]=='282330.KS']

,sector,longBusinessSummary,city,phone,country,companyOfficers,website,maxAge,address1,industry,...,regularMarketPrice,preMarketPrice,logo_url,fullTimeEmployees,zip,fax,underlyingSymbol,underlyingExchangeSymbol,headSymbol,uuid
0,Consumer Defensive,"BGF retail CO., LTD. operates convenience stor...",Seoul,82 1 577 8007,South Korea,[],https://www.bgfretail.com,1,BGF Building,Grocery Stores,...,178500,None,https://logo.clearbit.com/bgfretail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
financial_engineering_days = 252
end_date = datetime.today().date()
str_end_date = datetime.strftime(end_date, "%Y-%m-%d")
start_date = (end_date - timedelta(days=financial_engineering_days))
str_start_date = datetime.strftime(start_date, "%Y-%m-%d")
print(str_start_date)
print(str_end_date)


2021-08-03
2022-04-12


In [184]:
yf.Ticker("282330.KS").history(start=str_start_date, end=str_end_date, auto_adjust=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-08-03,159740.0,163170.0,158760.0,162680.0,18501,0.0,0
2021-08-04,160720.0,167580.0,160720.0,166600.0,46309,0.0,0
2021-08-05,166600.0,167580.0,165130.0,166600.0,25632,0.0,0
2021-08-06,169540.0,173950.0,168560.0,170520.0,62355,0.0,0
2021-08-09,169050.0,172480.0,166600.0,168070.0,27521,0.0,0
...,...,...,...,...,...,...,...
2022-04-05,179000.0,181000.0,177500.0,180000.0,21909,0.0,0
2022-04-06,180000.0,180500.0,177500.0,179500.0,24955,0.0,0
2022-04-07,177000.0,181500.0,177000.0,178000.0,30823,0.0,0


In [201]:
financial_engineering_days = 252
end_date = datetime.today().date()
str_end_date = datetime.strftime(end_date, "%Y-%m-%d")
start_date = (end_date - timedelta(days=financial_engineering_days))
str_start_date = datetime.strftime(start_date, "%Y-%m-%d")

for stock in market_stock_dict["kospi"]:
    symbol = stock+".KS"
    print(symbol)
    datetime.today().date()
    df = yf.Ticker(symbol).history(interval="1d", start = str_start_date, end = str_end_date, auto_adjust=True).reset_index()
    df = df[df["Volume"]>0].reset_index(drop=True)
    config_ndays = 20
    
    # 수익률 계산하기
    df["daily_rtn"] = df["Close"].pct_change()  # 퍼센트 변화율
    df["st_rtn"] = (1 + df["daily_rtn"]).cumprod()  # 누적곱 계산함수 return cumulative product over a DataFrame or Series axis.
    gagr = (df.iloc[-1]['st_rtn'] ** (financial_engineering_days / len(df.index)) - 1) * 100

    df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
    df["sma"] = df["TP"].rolling(config_ndays).mean()
    df["mad"] = df["TP"].rolling(config_ndays).apply(lambda x: pd.Series(x).mad())
    df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])

    df["pre1day_cci"] = df["CCI"].shift(1)
    df["trade"] = None
    df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
    df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])

    df["total_buy_price"] = 0
    df["shares"] = 0
    df["buy_price"] = 0
    df["sell_price"] = 0
    df["revenue"] = 0
    df["rate"] = 0

    buy_price_list = []
    for i, x in df.iterrows():
        if x["trade"] == "buy":
            buy_price_list.append(x["Close"])
            df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
            df.loc[i, "shares"] = len(buy_price_list)
            df.loc[i, "buy_price"] = np.mean(buy_price_list)
        elif x["trade"] == None:
            if i > 0:
                df.loc[i, "shares"] = df.loc[i-1, "shares"]
                df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
                df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
        elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
            first_price = buy_price_list.pop(0)
            df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
            df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
            df.loc[i, "sell_price"] = df.loc[i, "Close"]
            df.loc[i, "shares"] = len(buy_price_list)
            if len(buy_price_list) > 0:
                df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
                df.loc[i, "buy_price"] = np.mean(buy_price_list)
            else:
                df.loc[i, "total_buy_price"] = 0
                df.loc[i, "buy_price"] = 0

    r_dict = {
        "symbol" : symbol,
        "date" : datetime.strftime(df.iloc[-1]["Date"], "%Y-%m-%d"),
        "trade_signal" : df.iloc[-1]["trade"],
        "price" : df.iloc[-1]["Close"],
        "remain_shares" : df.iloc[-1]["shares"],
        "holding_shares_buy_price" : df.iloc[-1]["buy_price"],
        "cci_rtn" : df[df["trade"] == "sell"].iloc[-1]["rate"],
        "buy_and_hold_rtn" : gagr
    }
    print(r_dict)
    break

282330.KS
{'symbol': '282330.KS', 'date': '2022-04-11', 'trade_signal': None, 'price': 182000.0, 'remain_shares': 4, 'holding_shares_buy_price': 149690.0, 'cci_rtn': 9.17436883568017, 'buy_and_hold_rtn': 19.333156453419953}


In [198]:
df[df]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_rtn,st_rtn,...,mad,CCI,pre1day_cci,trade,total_buy_price,shares,buy_price,sell_price,revenue,rate
155,2022-04-05,179000.0,181000.0,177500.0,180000.0,21909,0.0,0,0.005587,1.106467,...,4009.166667,80.509942,82.956827,None,598760.0,4,149690.0,0.0,0.0,0.0
156,2022-04-06,180000.0,180500.0,177500.0,179500.0,24955,0.0,0,-0.002778,1.103393,...,3681.666667,69.111212,80.509942,None,598760.0,4,149690.0,0.0,0.0,0.0
157,2022-04-07,177000.0,181500.0,177000.0,178000.0,30823,0.0,0,-0.008357,1.094173,...,3090.000000,58.252427,69.111212,None,598760.0,4,149690.0,0.0,0.0,0.0
158,2022-04-08,178500.0,180000.0,177000.0,178500.0,19684,0.0,0,0.002809,1.097246,...,2533.333333,46.052632,58.252427,None,598760.0,4,149690.0,0.0,0.0,0.0
159,2022-04-11,176500.0,182000.0,176500.0,182000.0,29050,0.0,0,0.019608,1.118761,...,2320.000000,85.488506,46.052632,None,598760.0,4,149690.0,0.0,0.0,0.0


In [195]:
df[df["trade"]]

SyntaxError: invalid syntax (<ipython-input-195-63c9403daaa3>, line 1)

In [192]:
df[df["rate"]>0]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_rtn,st_rtn,...,mad,CCI,pre1day_cci,trade,total_buy_price,shares,buy_price,sell_price,revenue,rate
149,2022-03-28,179500.0,180000.0,174000.0,176000.0,36554,0.0,0,-0.027624,1.081879,...,5750.0,64.541063,118.274875,sell,598760.0,4,149690.0,176000.0,14790.0,9.174369


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             22 non-null     datetime64[ns]
 1   Open             22 non-null     float64       
 2   High             22 non-null     float64       
 3   Low              22 non-null     float64       
 4   Close            22 non-null     float64       
 5   Volume           22 non-null     int64         
 6   Dividends        22 non-null     int64         
 7   Stock Splits     22 non-null     int64         
 8   daily_rtn        21 non-null     float64       
 9   st_rtn           21 non-null     float64       
 10  TP               22 non-null     float64       
 11  sma              3 non-null      float64       
 12  mad              3 non-null      float64       
 13  CCI              3 non-null      float64       
 14  pre1day_cci      2 non-null      float64    

In [173]:
df[df["sma"]>0]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_rtn,st_rtn,...,mad,CCI,pre1day_cci,trade,total_buy_price,shares,buy_price,sell_price,revenue,rate
19,2022-04-08,178500.0,180000.0,177000.0,178500.0,19684,0,0,0.002809,1.043860,...,2017.500000,37.725458,NaN,None,0,0,0,0,0,0
20,2022-04-11,176500.0,182000.0,176500.0,182000.0,29050,0,0,0.019608,1.064327,...,1853.333333,89.028777,37.725458,None,0,0,0,0,0,0
21,2022-04-12,180500.0,184000.0,177500.0,178500.0,31749,0,0,-0.019231,1.043860,...,1499.166667,82.638503,89.028777,None,0,0,0,0,0,0


In [168]:
df["sma"] = df["TP"].rolling(config_ndays).mean()
df["sma"]

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
5               NaN
6               NaN
7               NaN
8               NaN
9               NaN
10              NaN
11              NaN
12              NaN
13              NaN
14              NaN
15              NaN
16              NaN
17              NaN
18              NaN
19    177358.333333
20    177691.666667
21    178141.666667
Name: sma, dtype: float64

In [163]:
df[df["TP"]>0]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_rtn,st_rtn,...,mad,CCI,pre1day_cci,trade,total_buy_price,shares,buy_price,sell_price,revenue,rate
0,2022-03-14,175000.0,178500.0,171000.0,171000.0,54739,0,0,NaN,NaN,...,NaN,NaN,NaN,None,0,0,0,0,0,0
2,2022-03-16,171500.0,185500.0,170000.0,179000.0,99809,0,0,0.046784,1.046784,...,NaN,NaN,NaN,None,0,0,0,0,0,0
4,2022-03-18,175000.0,179500.0,174000.0,177500.0,41056,0,0,-0.008380,1.038012,...,NaN,NaN,NaN,None,0,0,0,0,0,0
6,2022-03-22,178500.0,180000.0,176000.0,179000.0,28943,0,0,0.008451,1.046784,...,NaN,NaN,NaN,None,0,0,0,0,0,0
7,2022-03-23,178500.0,181000.0,176500.0,180000.0,32657,0,0,0.005587,1.052632,...,NaN,NaN,NaN,None,0,0,0,0,0,0
8,2022-03-24,178000.0,182000.0,178000.0,180000.0,37137,0,0,0.000000,1.052632,...,NaN,NaN,NaN,None,0,0,0,0,0,0
9,2022-03-25,180000.0,182500.0,178500.0,181000.0,32543,0,0,0.005556,1.058480,...,NaN,NaN,NaN,None,0,0,0,0,0,0
10,2022-03-28,179500.0,180000.0,174000.0,176000.0,36554,0,0,-0.027624,1.029240,...,NaN,NaN,NaN,None,0,0,0,0,0,0
11,2022-03-29,177000.0,178500.0,171500.0,173000.0,34427,0,0,-0.017045,1.011696,...,NaN,NaN,NaN,None,0,0,0,0,0,0
12,2022-03-30,174000.0,176500.0,172000.0,173500.0,33881,0,0,0.002890,1.014620,...,NaN,NaN,NaN,None,0,0,0,0,0,0


In [ ]:
ilist = []
for stock in market_stock_dict["kospi"]:
    symbol = stock+".ks"
    ilist.append(yf.Ticker(symbol).info)

In [ ]:
for stock in market_stock_dict["kospi"]:
    

In [78]:
df = yf.Ticker('282330.ks').history(interval="1d", start="2021-01-01", end="2022-04-11").reset_index()
df["Symbol"] = "282330"

In [79]:

config_ndays = 20
df["TP"] = (df["High"] + df["Low"] + df["Close"]) / 3
df["sma"] = df["TP"].rolling(config_ndays).mean()
df["mad"] = df["TP"].rolling(config_ndays).apply(lambda x: pd.Series(x).mad())
df["CCI"] = (df["TP"] - df["sma"]) / (0.015 * df["mad"])

df["pre1day_cci"] = df["CCI"].shift(1)
df["trade"] = None
df["trade"] = np.where((df["CCI"] >= -100) & (df["pre1day_cci"] < -100), "buy", df["trade"])
df["trade"] = np.where((df["CCI"] <= 100) & (df["pre1day_cci"] > 100), "sell", df["trade"])

df["total_buy_price"] = 0
df["shares"] = 0
df["buy_price"] = 0
df["sell_price"] = 0
df["revenue"] = 0
df["rate"] = 0

buy_price_list = []
for i, x in df.iterrows():
    if x["trade"] == "buy":
        buy_price_list.append(x["Close"])
        df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
        df.loc[i, "shares"] = len(buy_price_list)
        df.loc[i, "buy_price"] = np.mean(buy_price_list)
    elif x["trade"] == None:
        if i > 0:
            df.loc[i, "shares"] = df.loc[i-1, "shares"]
            df.loc[i, "buy_price"] = df.loc[i-1, "buy_price"]
            df.loc[i, "total_buy_price"] = df.loc[i-1, "total_buy_price"]
    elif x["trade"] == "sell" and df.loc[i-1, "shares"] > 0:
        first_price = buy_price_list.pop(0)
        df.loc[i, "revenue"] = df.loc[i, "Close"] - first_price
        df.loc[i, "rate"] = df.loc[i, "revenue"] / first_price * 100
        df.loc[i, "sell_price"] = df.loc[i, "Close"]
        df.loc[i, "shares"] = len(buy_price_list)
        if len(buy_price_list) > 0:
            df.loc[i, "total_buy_price"] = np.sum(buy_price_list)
            df.loc[i, "buy_price"] = np.mean(buy_price_list)
        else:
            df.loc[i, "total_buy_price"] = 0
            df.loc[i, "buy_price"] = 0

r_dict = {
    "symbol" : df.iloc[-1]["Symbol"],
    "date" : df.iloc[-1]["Date"],
    "trade_signal" : df.iloc[-1]["trade"],
    "price" : df.iloc[-1]["Close"]
}

In [98]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'Symbol', 'TP', 'sma', 'mad', 'CCI', 'pre1day_cci',
       'trade', 'total_buy_price', 'shares', 'buy_price', 'sell_price',
       'revenue', 'rate', 'return', 'acc_return'],
      dtype='object')

In [102]:
df[~df["trade"].isnull()][["Symbol", "Date", "trade", "shares", "buy_price", "sell_price", "total_buy_price", "revenue", "rate"]]

,Symbol,Date,trade,shares,buy_price,sell_price,total_buy_price,revenue,rate
21,282330,2021-02-02,sell,0,0.000000,0.0,0.0,0.0,0.000000
30,282330,2021-02-17,sell,0,0.000000,0.0,0.0,0.0,0.000000
39,282330,2021-03-03,buy,1,166110.000000,0.0,166110.0,0.0,0.000000
44,282330,2021-03-10,sell,0,0.000000,167580.0,0.0,1470.0,0.884956
49,282330,2021-03-17,sell,0,0.000000,0.0,0.0,0.0,0.000000
60,282330,2021-04-01,buy,1,159250.000000,0.0,159250.0,0.0,0.000000
72,282330,2021-04-19,buy,2,157780.000000,0.0,315560.0,0.0,0.000000
82,282330,2021-05-03,buy,3,155166.666667,0.0,465500.0,0.0,0.000000
93,282330,2021-05-20,sell,2,153125.000000,174440.0,306250.0,15190.0,9.538462
107,282330,2021-06-09,sell,1,149940.000000,181300.0,149940.0,24990.0,15.987461


In [145]:
df['revenue'].sum() / df[df["trade"]=="buy"]["buy_price"].sum()

KeyError: 'revenue'

In [87]:
def returns(book):
    # 손익 계산
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        # long 진입
        if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':
            buy = book.loc[i, 'adjclose']
            print('진입일 : ', i, 'long 진입 가격 : ', buy)
        # long 청산
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':
            sell = book.loc[i, 'adjclose']
            rtn = (sell - buy) / buy + 1 #  손익 계산
            book.loc[i, 'return'] = rtn
            print('청산일 : ', i, 'long 진입 가격 : ', buy, 'long 청산 가격 : ', sell, '| return : ', round(rtn, 4))
        if book.loc[i, 'trade'] == '':  # 제로 포지션
            buy = 0.0
            sell = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn =  acc_rtn * rtn  # 누적 수익률 계산
        book.loc[i, 'acc_return']  = acc_rtn
    print('Accumulated return : ', round(acc_rtn, 4))
    return (round(acc_rtn, 4))

In [91]:
for i in df.index:
    rtn = df.loc[i, 'return']
    print(rtn)
    break

1


In [88]:
returns(df)

Accumulated return :  1.0


1.0

In [86]:
df[~df["trade"].isnull()].head(20)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,TP,...,mad,CCI,pre1day_cci,trade,total_buy_price,shares,buy_price,sell_price,revenue,rate
21,2021-02-02,159740.0,161700.0,157290.0,158270.0,62919,0.0,0,282330,159086.666667,...,8943.316667,88.819895,120.458602,sell,0.0,0,0.000000,0.0,0.0,0.000000
30,2021-02-17,167580.0,170030.0,161700.0,163170.0,43092,0.0,0,282330,164966.666667,...,5821.200000,74.448186,135.401519,sell,0.0,0,0.000000,0.0,0.0,0.000000
39,2021-03-03,155820.0,167580.0,154350.0,166110.0,59126,0.0,0,282330,162680.000000,...,3100.066667,26.694766,-111.966635,buy,166110.0,1,166110.000000,0.0,0.0,0.000000
44,2021-03-10,171990.0,172480.0,162680.0,167580.0,40579,0.0,0,282330,167580.000000,...,3465.933333,90.794848,151.219512,sell,0.0,0,0.000000,167580.0,1470.0,0.884956
49,2021-03-17,169050.0,170030.0,165130.0,166110.0,29953,0.0,0,282330,167090.000000,...,3973.083333,55.498458,101.862707,sell,0.0,0,0.000000,0.0,0.0,0.000000
60,2021-04-01,155330.0,159250.0,154350.0,159250.0,33313,0.0,0,282330,157616.666667,...,4573.333333,-91.904762,-123.881537,buy,159250.0,1,159250.000000,0.0,0.0,0.000000
72,2021-04-19,156310.0,157780.0,154840.0,156310.0,24775,0.0,0,282330,156310.000000,...,1253.583333,-89.033659,-113.553114,buy,315560.0,2,157780.000000,0.0,0.0,0.000000
82,2021-05-03,151900.0,152880.0,149940.0,149940.0,46354,0.0,0,282330,150920.000000,...,3010.233333,-97.305118,-173.652695,buy,465500.0,3,155166.666667,0.0,0.0,0.000000
93,2021-05-20,174440.0,178360.0,171500.0,174440.0,37376,0.0,0,282330,174766.666667,...,10773.466667,76.511017,109.425436,sell,306250.0,2,153125.000000,174440.0,15190.0,9.538462
107,2021-06-09,181300.0,185220.0,180810.0,181300.0,23712,0.0,0,282330,182443.333333,...,3836.700000,91.102597,131.410256,sell,149940.0,1,149940.000000,181300.0,24990.0,15.987461


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Columns: 156 entries, sector to uuid
dtypes: bool(2), float64(90), int64(18), object(46)
memory usage: 241.1+ KB


In [2]:
def _get_kr_stock_info():
    r_data = []
    params = {
        "ServiceKey" : apisdata["getstockinfo"]["decoding_key"],
        "pageNo" : 1,
        "numOfRows" : 9999,
        "resultType" : "json"
        }
    url = apisdata["getstockinfo"]["request_url"]
    response = requests.get(url, params=params)
    if response.ok:
        get_dict = json.loads(response.text)
        r_data = get_dict["response"]["body"]["items"]["item"]

    r_df = pd.DataFrame(r_data)

    return r_df
    

In [30]:
df = _get_kr_stock_info()
# 상장폐지(lstgAbolDt) 안되었고, 예탁취소(dpsgCanDt)도 안된 종목만 본다.
f_df = df[(df["lstgAbolDt"] == '') & (df["dpsgCanDt"] == '')]

# 보통주
common_df = f_df[f_df["scrsItmsKcdNm"]=="보통주"]
common_df


,basDt,crno,isinCd,stckIssuCmpyNm,isinCdNm,scrsItmsKcd,scrsItmsKcdNm,stckParPrc,issuStckCnt,lstgDt,lstgAbolDt,dpsgRegDt,dpsgCanDt,issuFrmtClsfNm
0,20200423,0000000000000,HK0000214814,헝셩그룹,헝셩그룹,0101,보통주,0,80000000,20160818,,20190913,,전자증권
2,20200423,0000000046295,HK0000449303,윙입푸드홀딩스,윙입푸드홀딩스,0101,보통주,0,47973428,20181130,,20190913,,전자증권
3,20200423,0000000046622,HK0000452596,주이카쿠,주이카쿠,0101,보통주,1,0,,,,,예탁증권
4,20200423,0000000266537,KYG2115T1076,차이나크리스탈신소재홀딩스,차이나크리스탈신소재홀딩스,0101,보통주,.5,67783852,20160128,,20190913,,전자증권
8,20200423,0011001068885,KR8392070007,SBI 핀테크솔루션즈,SBI 핀테크솔루션즈,0101,보통주,0,24656540,20121217,,20190913,,전자증권
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,20200423,1801110162585,KR7058400003,케이엔엔,케이엔엔,0101,보통주,500,132429720,20101103,,20190913,,전자증권
9995,20200423,1801110170223,KR7102630001,고려자동화,고려자동화,0101,보통주,10000,173333,,,,,예탁증권
9996,20200423,1801110172667,KR7065570004,삼영이엔씨,삼영이엔씨,0101,보통주,500,8800000,20030121,,20190913,,전자증권
9997,20200423,1801110181428,KR7060370004,케이티서브마린,케이티서브마린,0101,보통주,1000,21900000,20020215,,20190913,,전자증권


In [ ]:
df.head()

In [ ]:
df

In [4]:
df[df["stckIssuCmpyNm"].str.contains("삼성")]

,basDt,crno,isinCd,stckIssuCmpyNm,isinCdNm,scrsItmsKcd,scrsItmsKcdNm,stckParPrc,issuStckCnt,lstgDt,lstgAbolDt,dpsgRegDt,dpsgCanDt,issuFrmtClsfNm
59,20200423,1101110005078,KR7000810002,삼성화재해상보험,삼성화재해상보험,0101,보통주,500,47374837,19900410,,20190913,,전자증권
60,20200423,1101110005078,KR7000811000,삼성화재해상보험,삼성화재해상보험1우,0201,우선주,500,3192000,19900410,,20190913,,전자증권
71,20200423,1101110005953,KR7032830002,삼성생명보험,삼성생명보험,0101,보통주,500,200000000,20100512,,20190913,,전자증권
135,20200423,1101110015762,KR7028260008,삼성물산(구 제일모직),삼성물산,0101,보통주,100,189690043,20150915,,20190913,,전자증권
136,20200423,1101110015762,KR702826K016,삼성물산(구 제일모직),삼성물산1우,0201,우선주,100,1627440,20150915,,20190913,,전자증권
249,20200423,1101110032568,KR7001360007,삼성제약,삼성제약,0101,보통주,500,61549713,19750704,,20190913,,전자증권
282,20200423,1101110037774,KR7003400009,삼성신약,삼성신약,0101,보통주,5000,40137314,19890531,19941013,19890531,19941015,예탁증권
454,20200423,1101110113475,KR7029760006,삼성상호신용금고,삼성상호신용금고,0101,보통주,10000,0,,,,,예탁증권
588,20200423,1101110168595,KR7010140002,삼성중공업,삼성중공업,0101,보통주,5000,630000000,19941122,,20190913,,전자증권
589,20200423,1101110168595,KR7010141000,삼성중공업,삼성중공업1우,0201,우선주,5000,114845,19941122,,20190913,,전자증권


In [ ]:
kospi_tickers = fdr.StockListing("KOSPI")["Symbol"].tolist()
kospi_tickers

In [ ]:
for ticker in kospi_tickers:
    df = fdr.DataReader(ticker)
    df.head()
    break

In [ ]:
df

In [ ]:
datetime.strptime("09:00:00", "%H:%M:%S").time()

In [ ]:
datetime.combine((datetime.today() + timedelta(days=1)), datetime.strptime("09:00", "%H:%M").time())

In [ ]:
datetime.now().time().hour

In [ ]:
if datetime.now().time() < time(hour=15, minute=9):
    print("aaa")